In [2]:
!pip install transformers
!pip install wandb
!pip install trl
!pip install pandas
!pip install datasets
!pip install accelerate
!pip install tyro
!pip install nltk -U

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 2.2.1 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable


  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

sentry-sdk 1.43.0 requires urllib3>=1.26.11; python_version >= "3.6", but you'll have urllib3 1.25.11 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: nltk in ./.local/lib/python3.8/site-packages (3.8.1)


In [3]:
import torch
from tqdm import tqdm
import pandas as pd
import wandb
import os

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

In [4]:
config = PPOConfig(
    model_name    = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
    learning_rate = 1.41e-5,
    ## log_with      = "wandb",
)

sent_kwargs = {
         "return_all_scores": True,
         "function_to_apply": "none",
         "batch_size": 16
}

In [5]:
## wandb.init()

wandb.init(mode="disabled")
os.environ['WANDB_DISABLED'] = 'true'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


# **Load 'financial_phrasebank' dataset**

Polar sentiment dataset of sentences from financial news. The dataset consists of 4840 sentences from English language financial news categorised by sentiment. The dataset is divided by agreement rate of 5-8 annotators.


# **Visualize details of dataset**

In [6]:
dataset_name="financial_phrasebank"

In [7]:
ds  = load_dataset(dataset_name,'sentences_50agree', split = "train")

In [8]:
ds

Dataset({
    features: ['sentence', 'label'],
    num_rows: 4846
})

In [9]:
ds[15:18]

{'sentence': ['Consolidated net sales increased 16 % to reach EUR74 .8 m , while operating profit amounted to EUR0 .9 m compared to a loss of EUR0 .7 m in the prior year period .',
  'Foundries division reports its sales increased by 9.7 % to EUR 63.1 mn from EUR 57.5 mn in the corresponding period in 2006 , and sales of the Machine Shop division increased by 16.4 % to EUR 41.2 mn from EUR 35.4 mn in the corresponding period in 2006 .',
  "HELSINKI ( AFX ) - Shares closed higher , led by Nokia after it announced plans to team up with Sanyo to manufacture 3G handsets , and by Nokian Tyres after its fourth-quarter earnings report beat analysts ' expectations , dealers said ."],
 'label': [2, 2, 2]}

In [10]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

In [11]:
def show_random_elements(dataset, num_examples=20):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []

    for _ in range( num_examples ):

        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame( dataset[picks] )        ## indexing 10 picks

    print(df)
    print(dataset.features.items())

    for column, typ in dataset.features.items():
        print(column)
        print(typ)
        print(ClassLabel)
        ## The isinstance() function returns True if the specified object
        ## is of the specified type, otherwise False
        if isinstance(typ, ClassLabel):
            print("Hello")
            df[column] = df[column].transform(lambda i: typ.names[i])
            ## print(typ.names[i])

    display(HTML(df.to_html()))

In [12]:
show_random_elements(ds)

                                             sentence  label
0   Incap Corporation Stock Exchange Announcement ...      1
1   Finnish Bank of +àland reports its operating p...      0
2   Stichting Pensioenfonds ABP : 4 118 122 shares...      1
3   Simultaneously , Alma Media has purchased a 35...      1
4   Profit after taxes was EUR 0.1 mn , compared t...      2
5   Glaston , headquartered in Tampere , Finland ,...      2
6   The hull of the vessel was built one block at ...      1
7   He also mentions that this improvement to the ...      2
8   Finnish real estate investor Sponda Plc said o...      1
9   Protalix is developing genetically engineered ...      1
10  The 5,000 megawatt wind farm being planned in ...      1
11  She will succeed Krister Kylas , who has decid...      1
12  The other seats would go to Edgar Edmonds , an...      1
13  Progress Group , QPR 's representative in Saud...      2
14  Tallink claims the watertight doors of both Va...      2
15  The above mentioned 

,sentence,label
0,"Incap Corporation Stock Exchange Announcement 29 April 2010 at 1 p.m. INVITATION TO A NEWS CONFERENCE ON INCAP 'S INTERIM REPORT Q1-2010 Incap will publish its interim report for January-March 2010 on Wednesday , 5 May 2010 .",neutral
1,Finnish Bank of +àland reports its operating profit fell to EUR 4.9 mn in the third quarter of 2007 from EUR 5.6 mn in the third quarter of 2006 .,negative
2,Stichting Pensioenfonds ABP : 4 118 122 shares representing 5.19 % of the share capital and voting rights .,neutral
3,"Simultaneously , Alma Media has purchased a 35 % share of Arena Interactive , a subsidiary of Arena Partners with a focus on mobile solutions development .",neutral
4,"Profit after taxes was EUR 0.1 mn , compared to EUR -0.4 mn the previous year .",positive
5,"Glaston , headquartered in Tampere , Finland , is a growing , international glass technology company .",positive
6,The hull of the vessel was built one block at a time and Ruukki delivered the plate material for each block as construction progressed .,neutral
7,He also mentions that this improvement to the service follows the recent expansion of the Finnlines service from Bilbao via Antwerp and Helsinki and from Hull via Helsinki to St. Petersburg .,positive
8,"Finnish real estate investor Sponda Plc said on Wednesday 12 March that it has signed agreements with Danske Bank A-S , Helsinki Branch for a 7-year EUR150m credit facility and with Ilmarinen Mutual Pension Insurance Company for a 7-year EUR50m credit facility .",neutral
9,Protalix is developing genetically engineered proteins from plant cells .,neutral


In [13]:
# ds = ds.rename_columns({"text": "sentence"})
ds = ds.filter(lambda x: len(x["sentence"]) > 200, batched=False)

In [14]:
ds

Dataset({
    features: ['sentence', 'label'],
    num_rows: 625
})

In [15]:
tokenizer           = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token

In [16]:
def tokenize( sample ):
    sample["input_ids"] = tokenizer.encode( sample["sentence"]    )[: 20]
    sample["query"]     = tokenizer.decode( sample["input_ids"] )
    return sample


ds = ds.map(tokenize, batched=False)
ds

Dataset({
    features: ['sentence', 'label', 'input_ids', 'query'],
    num_rows: 625
})

In [17]:
ds[15:18]

{'sentence': ["Seppala 's revenue increased by 0.2 % to EUR10 .1 m. In Finland , revenue went down by 2.4 % to EUR6 .8 m , while sales abroad rose by 6.2 % to EUR3 .3 m. Sales increased in all the Baltic countries as well as in Russia and Ukraine .",
  "At the request of Finnish media company Alma Media 's newspapers , research manager Jari Kaivo-oja at the Finland Futures Research Centre at the Turku School of Economics has drawn up a future scenario for Finland 's national economy by using a model developed by the University of Denver .",
  "`` The new agreement is a continuation to theagreement signed earlier this year with the Lemminkainen Group , whereby Cramo acquired the entire construction machine fleet ofLemminkainen Talo Oy Ita - ja Pohjois Suomo , and signed asimilar agreement , '' said Tatu Hauhio , managing director ofCramo Finland ."],
 'label': [2, 1, 1],
 'input_ids': [[0,
   14696,
   3807,
   2331,
   128,
   29,
   903,
   1130,
   30,
   321,
   4,
   176,
   7606,


### **My own data**

In [18]:
# my_own_datasets = load_dataset("text", data_files={ "train": "path", "validation": "path"} )


In [19]:
# my_own_datasets

# **Now this for actual RLHF**

In [20]:
def build_dataset(
         config,
         dataset_name="financial_phrasebank",
         input_min_text_length=2,
         input_max_text_length=8
):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer           = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token

    # load imdb with datasets

    #ds = load_dataset(dataset_name, split="train")
    ds  = load_dataset(dataset_name, 'sentences_50agree', split = "train")

    #ds = ds.rename_columns({"text": "sentence"})
    ds = ds.filter(lambda x: len(x["sentence"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode( sample["sentence"]    )[: input_size()]
        sample["query"]     = tokenizer.decode( sample["input_ids"] )
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [21]:
dataset = build_dataset(config)
# dataset = load_dataset('financial_phrasebank', 'sentences_50agree')

In [22]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

# **Load pre-trained GPT2 language models**
We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [23]:
model     = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

In [24]:
ppo_trainer = PPOTrainer(
                 config,
                 model,
                 ref_model,
                 tokenizer,
                 dataset=dataset,
                 data_collator=collator
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


# **Load BERT classifier (Reward Function)**
We load a BERT classifier fine-tuned on the IMDB dataset.

In [25]:
torch.cuda.is_available()

False

In [27]:
# torch.cuda.get_device_name()

In [28]:
device = ppo_trainer.accelerator.device
device

device(type='cpu')

In [29]:
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug

device

'cpu'

In [30]:
sentiment_pipe = pipeline("sentiment-analysis", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", device=device)

In [31]:
text = "Operating profit totaled EUR 9.4 mn , down from EUR 11.7 mn in 2004 ."   

sentiment_pipe(text, **sent_kwargs)

/home/chavan7/.local/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'negative', 'score': 4.749904155731201},
  {'label': 'neutral', 'score': -2.4718210697174072},
  {'label': 'positive', 'score': -2.789074420928955}]]

In [32]:
text = "In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn ."  

sentiment_pipe(text, **sent_kwargs)

[[{'label': 'negative', 'score': -2.473881244659424},
  {'label': 'neutral', 'score': -3.440216302871704},
  {'label': 'positive', 'score': 6.057607650756836}]]

In [33]:
text = "Pharmaceuticals group Orion Corp reported a fall in its third-quarter earnings that were hit by larger expenditures on R&D and marketing ."  

sentiment_pipe(text, **sent_kwargs)

[[{'label': 'negative', 'score': 4.775065898895264},
  {'label': 'neutral', 'score': -2.3186452388763428},
  {'label': 'positive', 'score': -2.8229196071624756}]]

In [34]:
text = "When this investment is in place , Atria plans to expand into the Moscow market ."  

sentiment_pipe(text, **sent_kwargs)

[[{'label': 'negative', 'score': -4.264527320861816},
  {'label': 'neutral', 'score': 4.1889967918396},
  {'label': 'positive', 'score': 0.7380496859550476}]]

# **Generation settings**
For the response generation we just use sampling and make sure top-k and nucleus sampling are turned off as well as a minimal length.

In [35]:
gen_kwargs = {
         "min_length":   -1,
         "top_k":       0.0,
         "top_p":       1.0,
         "do_sample":  True,
         "pad_token_id": tokenizer.eos_token_id
}

# **Optimize model**
### **Training loop**
The training loop consists of the following main steps:

*   Get the query and responses from the policy network (GPT-2)
*   Get sentiments for query/responses from BERT
*   Optimize policy with PPO using the (query, response, reward) triplet

In [36]:
output_min_length     = 4
output_max_length     = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)

In [37]:
generation_kwargs = {
    "min_length":     -1,
    "top_k":         0.0,
    "top_p":         1.0,
    "do_sample":    True,
    "pad_token_id": tokenizer.eos_token_id,
}

In [38]:
## ppo_trainer.config.steps = 100    ## 20,000
ppo_trainer.config.steps

20000

In [39]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    print(query_tensors)
    print(len(query_tensors))
    if epoch == 1:
        break

1it [00:00, 11.19it/s]

[tensor([    0,   597,  5246,  1173, 24753, 18419]), tensor([   0,  133, 9368,  128,   29, 8610]), tensor([    0, 14836]), tensor([   0,  597, 5246]), tensor([   0, 1640, 4516,  510,  491, 4839,  111]), tensor([  0, 597]), tensor([    0, 17986]), tensor([  0, 133]), tensor([   0,  133, 3457, 2156, 2964]), tensor([   0,  597, 5246, 1173, 3296]), tensor([   0,  250, 4779, 2186]), tensor([    0, 14229,     5,   375,  2202,    24,    34]), tensor([    0,   725, 16416, 23617,   100]), tensor([    0, 15685,   128,    29,   299, 11255,  6738]), tensor([   0, 1640, 4516,  510,  491, 4839]), tensor([    0, 41858,   138,  4832,   255,  5810]), tensor([   0, 5320, 4330, 2271, 2527]), tensor([  0, 133]), tensor([   0, 2606,  510, 2926]), tensor([    0, 24476,   168,  7236]), tensor([    0,   133,  4069,     9,    70,   518, 10298]), tensor([    0,   133,   786,    12, 13139]), tensor([    0,   597,  5246,  1173, 11767,  2643,  6596]), tensor([    0, 41024, 18366,   468]), tensor([    0, 31161]), t

In [40]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    print(epoch)
    print(batch)
    print('*********************')
    print('*********************')
    print('*********************')
    print('*********************')
    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len                             = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response                            = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append( response.squeeze()[-gen_len:] )
    batch["response"] = [ tokenizer.decode(r.squeeze()) for r in response_tensors ]
    print(batch)
    if epoch == 1:
        break

0it [00:00, ?it/s]

0
{'label': [tensor(2), tensor(1), tensor(2), tensor(0), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(2), tensor(2), tensor(0), tensor(1), tensor(1), tensor(2), tensor(2), tensor(1), tensor(2), tensor(2), tensor(2), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(0), tensor(2), tensor(1), tensor(1), tensor(1), tensor(2), tensor(2), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(0), tensor(2), tensor(0), tensor(1), tensor(2), tensor(2), tensor(1), tensor(2), tensor(2), tensor(1), tensor(1), tensor(2), tensor(2), tensor(0), tensor(1), tensor(0), tensor(0), tensor(1), tensor(2), tensor(1), tensor(2), tensor(1), tensor(1), tensor(2), tensor(2), tensor(1), tensor(1), tensor(2), tensor(0), tensor(1), tensor(1), tensor(1), tensor(2), tensor(1), tensor(2), tensor(1), tensor(2), tensor(0), tensor(1), tensor(1), tensor(0), tensor(1), tensor(1), tensor(1), tensor(2), tensor(1), tensor(2), tensor(2), tensor(1), tensor(0), tensor(1

1it [02:18, 138.83s/it]

{'label': [tensor(2), tensor(1), tensor(2), tensor(0), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(2), tensor(2), tensor(0), tensor(1), tensor(1), tensor(2), tensor(2), tensor(1), tensor(2), tensor(2), tensor(2), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(0), tensor(2), tensor(1), tensor(1), tensor(1), tensor(2), tensor(2), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(0), tensor(2), tensor(0), tensor(1), tensor(2), tensor(2), tensor(1), tensor(2), tensor(2), tensor(1), tensor(1), tensor(2), tensor(2), tensor(0), tensor(1), tensor(0), tensor(0), tensor(1), tensor(2), tensor(1), tensor(2), tensor(1), tensor(1), tensor(2), tensor(2), tensor(1), tensor(1), tensor(2), tensor(0), tensor(1), tensor(1), tensor(1), tensor(2), tensor(1), tensor(2), tensor(1), tensor(2), tensor(0), tensor(1), tensor(1), tensor(0), tensor(1), tensor(1), tensor(1), tensor(2), tensor(1), tensor(2), tensor(2), tensor(1), tensor(0), tensor(1),

1it [04:44, 284.52s/it]

{'label': [tensor(0), tensor(0), tensor(1), tensor(1), tensor(2), tensor(1), tensor(1), tensor(1), tensor(2), tensor(2), tensor(2), tensor(1), tensor(1), tensor(2), tensor(1), tensor(1), tensor(0), tensor(2), tensor(1), tensor(2), tensor(1), tensor(2), tensor(2), tensor(1), tensor(2), tensor(1), tensor(2), tensor(0), tensor(2), tensor(1), tensor(0), tensor(1), tensor(1), tensor(2), tensor(1), tensor(1), tensor(1), tensor(2), tensor(2), tensor(2), tensor(1), tensor(1), tensor(1), tensor(2), tensor(1), tensor(0), tensor(2), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(1), tensor(2), tensor(1), tensor(0), tensor(1), tensor(2), tensor(1), tensor(0), tensor(1), tensor(1), tensor(2), tensor(2), tensor(1), tensor(2), tensor(1), tensor(0), tensor(1), tensor(1), tensor(1), tensor(1), tensor(2), tensor(2), tensor(0), tensor(2), tensor(2), tensor(2), tensor(0), tensor(2), tensor(1), tensor(2), tensor(2), tensor(2), tensor(2), tensor(1), tensor(1), tensor(2), tensor(1), tensor(2),

In [41]:
batch.keys()

dict_keys(['label', 'input_ids', 'query', 'response'])

## **Compute sentiment score**

In [42]:
batch["query"]

['<s>Nordic',
 '<s>have',
 '<s>`` We have',
 '<s>Karachi, Sept',
 '<s>Finnish consumer',
 '<s>Incap Corporation Stock Exchange Release',
 '<s>Risk exposure by Non-',
 '<s>Under the agreement Benef',
 '<s>According to',
 '<s>NAV',
 '<s>``',
 '<s>Music is provided',
 '<s>In the Czech',
 '<s>KESKO CORPORATION',
 '<s>Marimekko makes',
 '<s>YIT',
 '<s>The international electronic industry company',
 '<s>In',
 '<s>Mr. Atul Chopra',
 '<s>SINGAP',
 '<s>HUHTAMAKI',
 '<s>The Finnish government announced',
 '<s>M',
 '<s>Products include Consumer',
 '<s>F',
 '<s>Retail chain',
 '<s>Finnish electronics',
 '<s>Performance in 2006 was',
 '<s>The GeoSolutions',
 '<s>wins 98 % acceptance 23',
 '<s>( AD',
 '<s>The company serves customers',
 '<s>We now have both the opportunity',
 '<s>Ju',
 '<s>The aforementioned shareholders have informed',
 '<s>`` Of',
 '<s>Based',
 '<s>Finn',
 '<s>Finn',
 '<s>15',
 '<s>A total EUR73.7',
 '<s>H',
 '<s>Ald',
 '<s>7 March 2011 -',
 '<s>The solution is demonstrated on a'

In [43]:
batch["response"]

[' junction interchangeable interchangeable switches flaw immobil',
 'Loop elbows insider midway insider insider insider',
 ' widen],"CLOSE latitude nowhere nowhere nowhere Legisl intrigue latitude autop subter narrowEEP apart',
 ' longstanding KEY tailor shortcut nowhere latitudences',
 ' insider interchangeable anonymity eve interchangeable geographically',
 ' rearr maneuvermeg brazenrupulous cunningoks',
 'wk MUCH gestation Parkway antibiotic Included urinary coy',
 'oksennes infinite elevation barrier',
 ' enorm scenicpit intric insider anonymity',
 ' Mov engulfArtist latituderup× conce runway engulf Slightly Parkway Oval uncanny spiritually nowhere',
 'ß runway runway Somewhere curvllular Mur eaves widening epoch Alleg widen Alleg',
 ' grop categor shadowoks setup extended gulf precedEEP flaw Tripept.), latitude',
 ' infinite fielderrupulous lane Somewhere',
 ' presses grop Sneak Sneak spared Halhun Hal',
 " bona citation jumper:'function obfusc Swap insider SomethingITCH autop in

In [44]:
texts = [ q + r for q, r in zip(batch["query"], batch["response"]) ]

In [45]:
texts

['<s>Nordic junction interchangeable interchangeable switches flaw immobil',
 '<s>haveLoop elbows insider midway insider insider insider',
 '<s>`` We have widen],"CLOSE latitude nowhere nowhere nowhere Legisl intrigue latitude autop subter narrowEEP apart',
 '<s>Karachi, Sept longstanding KEY tailor shortcut nowhere latitudences',
 '<s>Finnish consumer insider interchangeable anonymity eve interchangeable geographically',
 '<s>Incap Corporation Stock Exchange Release rearr maneuvermeg brazenrupulous cunningoks',
 '<s>Risk exposure by Non-wk MUCH gestation Parkway antibiotic Included urinary coy',
 '<s>Under the agreement Benefoksennes infinite elevation barrier',
 '<s>According to enorm scenicpit intric insider anonymity',
 '<s>NAV Mov engulfArtist latituderup× conce runway engulf Slightly Parkway Oval uncanny spiritually nowhere',
 '<s>``ß runway runway Somewhere curvllular Mur eaves widening epoch Alleg widen Alleg',
 '<s>Music is provided grop categor shadowoks setup extended gulf p

In [46]:
pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
pipe_outputs

[[{'label': 'negative', 'score': -3.076402187347412},
  {'label': 'neutral', 'score': 6.228089332580566},
  {'label': 'positive', 'score': -2.307486057281494}],
 [{'label': 'negative', 'score': -2.7645912170410156},
  {'label': 'neutral', 'score': 6.335371971130371},
  {'label': 'positive', 'score': -2.8783717155456543}],
 [{'label': 'negative', 'score': -2.770263671875},
  {'label': 'neutral', 'score': 6.025016784667969},
  {'label': 'positive', 'score': -2.657650947570801}],
 [{'label': 'negative', 'score': -2.8166840076446533},
  {'label': 'neutral', 'score': 6.564802646636963},
  {'label': 'positive', 'score': -3.1765313148498535}],
 [{'label': 'negative', 'score': -2.7812907695770264},
  {'label': 'neutral', 'score': 6.60267972946167},
  {'label': 'positive', 'score': -3.0964577198028564}],
 [{'label': 'negative', 'score': -2.553628444671631},
  {'label': 'neutral', 'score': 6.342000961303711},
  {'label': 'positive', 'score': -3.199592113494873}],
 [{'label': 'negative', 'score':

In [47]:
rewards = [ torch.tensor(output[1]["score"]) for output in pipe_outputs]
rewards

[tensor(6.2281),
 tensor(6.3354),
 tensor(6.0250),
 tensor(6.5648),
 tensor(6.6027),
 tensor(6.3420),
 tensor(6.5203),
 tensor(6.6561),
 tensor(6.5438),
 tensor(5.8385),
 tensor(6.0423),
 tensor(6.6994),
 tensor(6.5540),
 tensor(2.3414),
 tensor(6.0802),
 tensor(6.6694),
 tensor(6.5247),
 tensor(6.4822),
 tensor(6.3585),
 tensor(6.3329),
 tensor(6.3951),
 tensor(6.3865),
 tensor(6.5736),
 tensor(6.5625),
 tensor(6.3614),
 tensor(6.5010),
 tensor(6.6032),
 tensor(6.5285),
 tensor(6.7909),
 tensor(6.6263),
 tensor(6.4232),
 tensor(6.5444),
 tensor(6.7891),
 tensor(6.7206),
 tensor(6.6772),
 tensor(6.3995),
 tensor(6.7337),
 tensor(6.2738),
 tensor(6.5040),
 tensor(6.4801),
 tensor(6.7547),
 tensor(6.2637),
 tensor(6.5575),
 tensor(6.5159),
 tensor(6.6569),
 tensor(6.4714),
 tensor(6.6318),
 tensor(6.5721),
 tensor(6.4898),
 tensor(6.5361),
 tensor(6.6424),
 tensor(6.4173),
 tensor(6.6741),
 tensor(6.4366),
 tensor(4.9894),
 tensor(6.5602),
 tensor(6.6610),
 tensor(6.1441),
 tensor(6.7505

In [48]:
len(rewards)

128

In [49]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    print(epoch)

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len                             = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response                            = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append( response.squeeze()[-gen_len:] )
    batch["response"] = [ tokenizer.decode(r.squeeze()) for r in response_tensors ]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [ torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(
                     query_tensors,
                     response_tensors,
                     rewards
    )
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:00, ?it/s]

0


/home/chavan7/.local/lib/python3.8/site-packages/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (93.66) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/home/chavan7/.local/lib/python3.8/site-packages/trl/trainer/ppo_trainer.py:1212: UserWarning: The average ratio of batch (367.75) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
1it [04:36, 276.00s/it]

1


2it [09:11, 275.56s/it]

2


3it [13:50, 277.19s/it]

3


4it [19:17, 289.39s/it]


In [65]:
# torch.cuda.get_device_name(0)

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher initial coefficient.

Let's inspect some examples from the IMDB dataset. We can use model_ref to compare the tuned model model against the model before optimisation.

In [66]:
#### get a batch from the dataset
bs                 = 16
game_data          = dict()

In [67]:
game_data

{}

In [68]:
dataset.set_format("pandas")

In [69]:
df_batch           = dataset[:].sample(bs)
df_batch

,sentence,label,input_ids,query
13,The world 's second largest stainless steel ma...,2,"[0, 133, 232]",<s>The world
179,"Finnish GeoSentric , a developer and provider ...",2,"[0, 597, 5246, 1173]",<s>Finnish
306,Finnish forest machinery and equipment maker P...,2,"[0, 597]",<s>F
620,have notified that as a result of the Company ...,0,"[0, 11990]",<s>have
387,Affecto will provide a new EFI Data Warehouse ...,1,"[0, 250, 45700, 139, 40]",<s>Affecto will
396,Aspocomp said it will spin off its Chinese and...,1,"[0, 1620, 642, 1975, 7474, 26]",<s>Aspocomp said
278,"In a release , the Company said that Vocollect...",2,"[0, 1121, 10, 800, 2156, 5]","<s>In a release, the"
204,The Innova 2 building will be located close to...,1,"[0, 133, 9548, 3900, 132]",<s>The Innova 2
518,The non-recurring costs caused to Talentum 's ...,1,"[0, 133, 786, 12, 13139]",<s>The non-rec
494,Incap Contract Manufacturing Services Pvt Ltd ...,1,"[0, 1121, 10906, 15739, 12047, 1820]",<s>Incap Contract Manufacturing Services


In [70]:
game_data["query"] = df_batch["query"].tolist()
query_tensors      = df_batch["input_ids"].tolist()

In [71]:
response_tensors_ref, response_tensors = [], []

In [72]:
#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()

    output  = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)


    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

In [73]:
#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"]  = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

In [74]:
#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

/home/chavan7/.local/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [75]:
texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

In [76]:
# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

,query,response (before),response (after),rewards (before),rewards (after)
0,<s>The world,Champions rings Klu KluVKills Klusets whatsil...,Vijzzo× latitude latitude smoot insiderEEP co...,6.427192,6.259383
1,<s>Finnish,Klu Klu PROT Banner Sounds binariessetssets K...,wiser Operator latitude angle SPACE]'rip jump...,6.578957,6.096363
2,<s>F,ear Flags COL Crestorningsspring Summercirc K...,ocitySEE unsc Parkway SEE Cuomo × sche latitud...,6.567821,6.319682
3,<s>have,winters Klu Volks Scout Klusets UNDER,KEYpiredrupulousrupulous bonaouveromers,6.541896,4.463653
4,<s>Affecto will,Klu peaksThumbnail Riderssetssets calfitles,widened Sneak insider RecallBUS culp foresee ...,6.634706,6.463746
5,<s>Aspocomp said,sets Paddock Bundes Brassetssetscollar Runs,refining infrared Experjoin Oval bona imperso...,6.591565,6.572432
6,"<s>In a release, the",pinegging Crimean Yuan Chron Antarctic padded...,Recall subterrupulous Cull avenue haun Ern gl...,6.659610,5.500474
7,<s>The Innova 2,cher hoselevard NVstrings,rupulous geographicallyoks calculated lane,6.674257,6.705620
8,<s>The non-rec,"kHz misconcuts."")), Sanctuary royaltiespotsil...",omers latitude indefinite shenanigans eaves Ni...,6.703739,6.568322
9,<s>Incap Contract Manufacturing Services,Ukrainians ear escapeschetpak noonshots origi...,Recall Parkway differentiate lapse anglescape...,6.696066,6.532852


In [77]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)    6.595707
rewards (after)     6.281386
dtype: float64


median:


rewards (before)    6.603034
rewards (after)     6.520298
dtype: float64

In [78]:
## model.save_pretrained(    "gpt2-imdb-pos-v2", push_to_hub=True)
## tokenizer.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=True)

model.save_pretrained(    "gpt2-imdb-pos-v2", push_to_hub=False)
tokenizer.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=False)

('gpt2-imdb-pos-v2/tokenizer_config.json',
 'gpt2-imdb-pos-v2/special_tokens_map.json',
 'gpt2-imdb-pos-v2/vocab.json',
 'gpt2-imdb-pos-v2/merges.txt',
 'gpt2-imdb-pos-v2/added_tokens.json',
 'gpt2-imdb-pos-v2/tokenizer.json')